In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 32
block_size = 128
max_iters = 3000 # number of iterations
learning_rate = 2e-5 # update model weights based on the derivative
eval_iters = 100 # number of iterations after which a print statement is executed
n_embd = 384 # how many features are captured
n_head = 4 # how many heads
n_layer = 4 # the number of decoded blocks
dropout = 0.2 # drop the number of neurons. In this case 20% of neurons.

print(device)

cuda


In [5]:
chars = ""
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [7]:
# K = Key
# Q = query
# V = value
# Dot product of Key, Query -> Scaling by 1/sqrt -> torch.tril (masking) -> softmax
# Matrix multiply the above with value
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # register the no look ahead masking -> just registers the current model state . This code is used to prevent overhead computation
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        # @ is matrix multiplication
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # As we advance through every step, the mask is removed and the value is exposed
        # [1, 0, 0]
        # [1, 0.6, 0]
        # [1, 0.6, 0.4]
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [8]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # number of heads or features that will be captured in the multi-head attention
        head_size = n_embd // n_head
        # self-attention
        self.sa = MultiHeadAttention(n_head, head_size)
        # feed forward
        self.ffwd = FeedFoward(n_embd)
        # post-norm (add and normalize) architecture
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # self attention
        y = self.sa(x)
        # add and norm - post-norm
        x = self.ln1(x + y)
        # feed forward
        y = self.ffwd(x)
        # add and norm - post-norm
        x = self.ln2(x + y)
        return x

In [9]:
# B = Batch
# T = Time
# F = Feature
# Since there are 4 heads we have 4 features per head
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    """ This function conctatenates the result of self-attention (Head class) and does a linear transformation (nn.Linear)"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        # process all the heads asyncrhronously. Thei is the opposite of Sequential where the next input is dependant on the previous output
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [10]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [11]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # Positional Embedding
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # The code below is to specify the number of decoders. Because n_layer=4, there will be 4 decoders.
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # Create the final linear layer after the n_layer number of decoders
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    # Weight initialization function
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        # The Decoder layer. This is irrespective of the number of decoders that we are using
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        # The layer where we do the linear transformation
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [21]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [25]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step: 0, train loss: 3.139, val loss: 3.145
step: 100, train loss: 3.141, val loss: 3.152
step: 200, train loss: 3.139, val loss: 3.150
step: 300, train loss: 3.144, val loss: 3.144
step: 400, train loss: 3.137, val loss: 3.150
step: 500, train loss: 3.142, val loss: 3.146
step: 600, train loss: 3.144, val loss: 3.150
step: 700, train loss: 3.140, val loss: 3.145
step: 800, train loss: 3.138, val loss: 3.148
step: 900, train loss: 3.142, val loss: 3.150
step: 1000, train loss: 3.142, val loss: 3.145
step: 1100, train loss: 3.138, val loss: 3.151
step: 1200, train loss: 3.149, val loss: 3.149
step: 1300, train loss: 3.143, val loss: 3.145
step: 1400, train loss: 3.143, val loss: 3.149
step: 1500, train loss: 3.142, val loss: 3.143
step: 1600, train loss: 3.140, val loss: 3.149
step: 1700, train loss: 3.142, val loss: 3.155
step: 1800, train loss: 3.138, val loss: 3.153
step: 1900, train loss: 3.144, val loss: 3.153
step: 2000, train loss: 3.141, val loss: 3.151
step: 2100, train loss: 3

In [23]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?Aifi lhhthh m eDta al 
gd tentrfrtswl  tf"it b a"epho ncro oscsdih;gd tolfhivA k 
eeher.iw" iae h dc
